In [30]:
#create the csv file of corpus and do preprocessing of documents

In [31]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from langdetect import detect

In [32]:
import string
import glob
import nltk
import math
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\siddh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [33]:
#case folding
def case_folding(lines):
  lines=lines.casefold()
  return lines
#removing punctuations
def rm_punctuation(words):
  words=words.translate(str.maketrans('','',string.punctuation))
  return words
#tkenization
def tokenization(words):
  token=nltk.word_tokenize(words)
  return token
#removing single letter tokens
def rm_single(token):
  for i in token:
    if len(i)==1:
      token.remove(i)
  return token
#removing stop words
def rm_stop_words(token):
  stop_words = set(stopwords.words('english'))
  for word in token:
    if word in stop_words:
        token.remove(word)
  return token
#stemming of tokens
def stemming(token):
  porter = PorterStemmer()
  for i in range(0,len(token)):
    token[i]=porter.stem(token[i])
  return token
#lemmatizing of tokens
def lemmatizing(token):
  for i in range(0,len(token)):
    token[i]=WordNetLemmatizer().lemmatize(token[i])
  return token

In [34]:
def pre_processing(text):
    text=case_folding(text)
    text=rm_punctuation(text)
    tokens=tokenization(text)
    tokens=rm_single(tokens)
    tokens=rm_stop_words(tokens)
    tokens=stemming(tokens)
    tokens=lemmatizing(tokens)
    return tokens

In [35]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

# load the model and the feature extraction object
with open('spam_model.pkl', 'rb') as file:
    model = pickle.load(file)

with open('vectorizer.pkl', 'rb') as file:
    feature_extraction = pickle.load(file)

# define new input text
new_input = ["I've been searching for the right words to thank you for this breather. I promise i wont take your help for granted and will fulfil my promise. You have been wonderful and a blessing at all times"]

# transform the new input text to feature vectors using the saved feature extraction object
new_input_features = feature_extraction.transform(new_input)

# make predictions on the new input using the trained model
predictions = model.predict(new_input_features)

if predictions[0] == 1:
    print('Ham mail')
else:
    print('Spam mail')


Ham mail


In [36]:
v={35: 'U.S. NEWS', 5: 'COMEDY', 22: 'PARENTING', 40: 'WORLD NEWS', 7: 'CULTURE & ARTS', 32: 'TECH', 28: 'SPORTS', 10: 'ENTERTAINMENT', 24: 'POLITICS', 37: 'WEIRD NEWS', 11: 'ENVIRONMENT', 9: 'EDUCATION', 6: 'CRIME', 27: 'SCIENCE', 38: 'WELLNESS', 3: 'BUSINESS', 30: 'STYLE & BEAUTY', 13: 'FOOD & DRINK', 20: 'MEDIA', 25: 'QUEER VOICES', 17: 'HOME & LIVING', 39: 'WOMEN', 2: 'BLACK VOICES', 34: 'TRAVEL', 21: 'MONEY', 26: 'RELIGION', 19: 'LATINO VOICES', 18: 'IMPACT', 36: 'WEDDINGS', 4: 'COLLEGE', 23: 'PARENTS', 1: 'ARTS & CULTURE', 29: 'STYLE', 15: 'GREEN', 31: 'TASTE', 16: 'HEALTHY LIVING', 33: 'THE WORLDPOST', 14: 'GOOD NEWS', 41: 'WORLDPOST', 12: 'FIFTY', 0: 'ARTS', 8: 'DIVORCE'}

In [37]:
query = "youtube social media instagram "
limit = 1000
tweets = [['Date', 'User', 'Tweet', 'Url', 'Followers', 'Likes', 'Retweets','Category', 'Spam/Ham', 'tokens']]
import pickle

with open('classify_model.pkl', 'rb') as f:
    category_model = pickle.load(f)

with open('spam_model.pkl', 'rb') as f:
    spam_model = pickle.load(f)

for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    # print(vars(tweet))
    # break
    if len(tweets) == limit:
        break
    else:
        if detect(tweet.content)=='en':
            spam_prediction = spam_model.predict(feature_extraction.transform([tweet.content]))
            if spam_prediction[0] == 1:
                spam_pre='Ham'
            else:
                spam_pre='Spam'
            category=v[category_model.predict([tweet.content])[0]]
            tokens=pre_processing(tweet.content)
            tweets.append([tweet.date, tweet.username, tweet.content,tweet.url, tweet.user.followersCount, tweet.likeCount, tweet.retweetCount,category,spam_pre,tokens])

df = pd.DataFrame(tweets, columns=['Date', 'User', 'Tweet','Url', 'Followers', 'Likes', 'Retweets','Category','Spam/Ham','tokens'])
print(df)

C:\Users\siddh\AppData\Local\Temp\ipykernel_7428\1684259422.py:18: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  if detect(tweet.content)=='en':
C:\Users\siddh\AppData\Local\Temp\ipykernel_7428\1684259422.py:19: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  spam_prediction = spam_model.predict(feature_extraction.transform([tweet.content]))
C:\Users\siddh\AppData\Local\Temp\ipykernel_7428\1684259422.py:24: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  category=v[category_model.predict([tweet.content])[0]]
C:\Users\siddh\AppData\Local\Temp\ipykernel_7428\1684259422.py:25: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tokens=pre_processing(tweet.content)
C:\Users\siddh\AppData\Local\Temp\ipykernel_7428\1684259422.py:26: DeprecatedFeatureWarning: username is deprecated, use user.username instead
  tweets.append([tweet.date, tweet.username, tweet.content,tweet.url, tweet.user.foll

                          Date             User  \
0                         Date             User   
1    2023-04-10 17:35:34+00:00  vitiligosociety   
2    2023-04-10 17:10:50+00:00      SojanTaz360   
3    2023-04-10 17:05:00+00:00    ScottGraffius   
4    2023-04-10 17:00:06+00:00      Garenthino1   
..                         ...              ...   
995  2023-03-30 14:21:41+00:00   smartedgeideas   
996  2023-03-30 14:09:24+00:00       banafahgeo   
997  2023-03-30 13:49:12+00:00  Learning_Movies   
998  2023-03-30 13:23:47+00:00    SarniaJournal   
999  2023-03-30 13:00:36+00:00  wakeuplegendary   

                                                 Tweet  \
0                                                Tweet   
1    Sonam, a beauty influencer from London, has bu...   
2    and analyzing data to track performance and ma...   
3    If you’d like to find out the lifespan (half-l...   
4    Q: Can I use social media ads to promote my Yo...   
..                                     

In [38]:
df.to_csv('corpus.csv', mode='a', index=False, header=False)

In [39]:
print(df.head())

                        Date             User  \
0                       Date             User   
1  2023-04-10 17:35:34+00:00  vitiligosociety   
2  2023-04-10 17:10:50+00:00      SojanTaz360   
3  2023-04-10 17:05:00+00:00    ScottGraffius   
4  2023-04-10 17:00:06+00:00      Garenthino1   

                                               Tweet  \
0                                              Tweet   
1  Sonam, a beauty influencer from London, has bu...   
2  and analyzing data to track performance and ma...   
3  If you’d like to find out the lifespan (half-l...   
4  Q: Can I use social media ads to promote my Yo...   

                                                 Url  Followers  Likes  \
0                                                Url  Followers  Likes   
1  https://twitter.com/vitiligosociety/status/164...       4157      0   
2  https://twitter.com/SojanTaz360/status/1645474...         10      0   
3  https://twitter.com/ScottGraffius/status/16454...       8358      1  